In [212]:
import urllib2, time, random, re
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import requests
import pickle
import nltk
import surprise
import math
from sklearn.metrics.pairwise import cosine_similarity
import re

In [2]:
#find all the links given website and append to all_url list. From that list find all the links that end with /boardgame/somedigits
soup = BeautifulSoup(urllib2.urlopen(urllib2.Request("https://boardgamegeek.com/browse/boardgame")).read())
all_url = []
for i in range(1,6):
    soup = BeautifulSoup(urllib2.urlopen(urllib2.Request("https://boardgamegeek.com/browse/boardgame/page/" + str(i))).read())
    for a in soup.find_all('a', href = True):
        all_url.append(a['href'])
board_games = []
for url in all_url:
    if re.match('/boardgame/\d+\w+', url):
        board_games.append(url)

/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [3]:
#get the board game ID numbers
board_games_nums =[]
for b in board_games:
    d = re.search('\d+', b)
    board_games_nums.append(d.group(0))

In [4]:
#find the descriptions of all the games and add to dictionary
games = list(set(board_games_nums))
dic ={}
while len(games)>0:
    print len(games)
    s = ','.join(games[:25])
    final_url = 'http://www.boardgamegeek.com/xmlapi/boardgame/' + s + '?comments=1'
    print final_url
    soup = BeautifulSoup(urllib2.urlopen(urllib2.Request(final_url)).read())
    for num in games[:25]:
        name_of_game = soup.find('boardgame', {'objectid': num}).find('name', {'primary' : 'true'}).text
        des = soup.find('boardgame', {'objectid': num}).description.text
        dic[name_of_game] = {}
        dic[name_of_game]['description'] = des
        #list_of_comments =[]
        #for c in soup.find('boardgame', {'objectid': num}).find_all('comment', {'usernames': True}):
        #    list_of_comments.append(c.text)
        #dic[name_of_game]['comments'] = ' '.join(list_of_comments)
    games = games[25:]


518
http://www.boardgamegeek.com/xmlapi/boardgame/134726,201808,9674,246744,139976,93,164928,68425,182028,21763,20963,37046,54043,205896,712,27833,155362,91312,42215,40834,82222,143741,43570,161970,205059?comments=1
493
http://www.boardgamegeek.com/xmlapi/boardgame/27173,155703,92415,175914,17226,95527,18098,162082,54138,42052,128271,157354,521,70323,43111,136888,164153,140934,154203,221965,73761,17133,31481,123260,231581?comments=1
468
http://www.boardgamegeek.com/xmlapi/boardgame/194607,146652,111799,128996,97207,125618,135219,2163,10630,31594,84876,244992,204305,22545,136063,101721,172308,209778,37111,103343,12333,127060,170416,73439,35677?comments=1
443
http://www.boardgamegeek.com/xmlapi/boardgame/20551,192135,91,35570,198994,94,150,3201,155821,153938,146278,102652,92828,93260,165722,100901,137269,555,168584,113924,12942,234,148228,43015,2346?comments=1
418
http://www.boardgamegeek.com/xmlapi/boardgame/31730,215341,172287,177736,175640,12493,209685,147020,21790,124361,9440,39351,1

In [17]:
#saving the dictionary 
pickle_out = open('dic_of_game_desp', 'wb')
pickle.dump(dic, pickle_out)
pickle_out.close()

In [18]:
#loading the dictionary
pickle_in = open('dic_of_game_desp', 'rb')
dic2 = pickle.load(pickle_in)

In [68]:
#tokenize words
#do some preprocessing with Lemmatizer and Stemming
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
tokenize ={}
tokenize_lem = {}
toeknize_stem = {}
stop_words = stopwords.words('english')
stop_words.append('br')
stop_words = set(stop_words)
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
for game in dic.keys():
    des = tokenizer.tokenize(dic[game]['description'])
    lower_des = [d.lower() for d in des]
    filtered_des = [w for w in lower_des if not w in stop_words]
    lem_des = [lemmatizer.lemmatize(w) for w in filtered_des]
    stem_des = [ps.stem(w) for w in filtered_des]
    tokenize_lem[game] = nltk.FreqDist(lem_des)
    tokenize[game]= nltk.FreqDist(filtered_des)
    tokenize_stem[game] = nltk.FreqDist(stem_des)

In [182]:
#change tokenize words to Dataframe
tokenize_df = pd.DataFrame.from_dict(tokenize, orient = 'index').fillna(0)
#tokenize_stem_df = pd.DataFrame.from_dict(tokenize_stem, orient = 'index').fillna(0)
#tokenize_lem_df = pd.DataFrame.from_dict(tokenize_lem, orient = 'index').fillna(0)

In [183]:
#Term frequency
def term_freq(df):
    df.loc[:,:]= df.loc[:,:].div(df.sum(axis=1), axis = 0)
    return(df)

In [184]:
def IDF(df):
    num_doc_with_word = df.astype(bool).sum(axis=0)
    num_games = len(df.index)
    idf = np.log(num_games / (1 + df.astype(bool).sum(axis=0)))
    return(idf)

In [185]:
def tf_idf(tf_df, idf):
    df = tf_df.multiply(idf, axis =1)
    return(df)

In [186]:
t = term_freq(tokenize_df)
idf_t = IDF(t)
t_dif = tf_idf(t, idf_t.values)

In [292]:
def recommend(game, df = t_dif):
    cos = cosine_similarity(df)
    names = [c.lower() for c in df.index.values]
    cos_df = pd.DataFrame(cos, index = names, columns = names)
    r = re.compile(str(game).lower())
    games = [w.lower() for w in list(t_dif.index.values)]
    g = filter(r.match, games)
    if len(g) == 0:
        print('This Game is not in the top 100 games in BGG')
    else:
        top = cos_df.loc[:,g[0]].sort_values(ascending = False)[1:11].index.values
        for c, value in enumerate(top, 1):
            print(c, value)

In [293]:
recommend('caverna')

(1, u'vast: the crystal caverns')
(2, u'summoner wars: guild dwarves vs cave goblins')
(3, u'agricola: all creatures big and small')
(4, u'agricola (revised edition)')
(5, u'agricola')
(6, u'ora et labora')
(7, u'above and below')
(8, u'summoner wars: alliances master set')
(9, u'belfort')
(10, u'the colonists')


In [295]:
recommend('Jens favorite game')

This Game is not in the top 100 games in BGG
